In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
# Load the dataset
df = pd.read_csv(r"C:\Users\amcys\Documents\02.Education\01.Online Education\03.DataTech Florida - Jerry Strazzeri\09.Trabalhos Finais\Walmart Project\Data\Final_database_V1.csv")
pd.set_option('display.max_columns', None)
df.head(5)

,date,order_id,order_amount,region,items_delivered,items_missing,delivery_hour,customer_id,customer_name,customer_age,driver_id,driver_name,driver_age,driver_trips,product_name,category,price
0,2023-02-12,b0b0ed0b-6d92-4747-a27d-32fa5b293d48,341.92,Altamonte Springs,19,0,04:02:37,WCID5359,Amanda Frank,48,WDID10634,Kevin Castillo DDS,65,48,NaN,NaN,NaN
1,2023-02-12,052ef301-b6a1-4e08-a6eb-8c92f64c11af,113.50,Clermont,8,0,17:58:42,WCID5000,Dustin Spencer,38,WDID10766,Heather Welch,60,25,NaN,NaN,NaN
2,2023-02-12,1c052a51-448f-4859-a739-7066ac41b8e8,442.30,Apopka,19,0,17:14:01,WCID5040,Amy Harrison,72,WDID10966,Shannon Reyes,43,24,NaN,NaN,NaN
3,2023-02-12,ca4bb404-bb71-40ee-925d-b816c97f4a9c,277.32,Clermont,13,0,03:52:42,WCID5867,Jose Jennings,85,WDID11100,Brett Young,48,39,NaN,NaN,NaN
4,2023-02-12,9cc29314-15ad-4259-9547-7b70177ed6de,462.46,Clermont,15,0,14:01:55,WCID6045,Zachary Campbell,60,WDID09899,Stephen Patton,18,47,NaN,NaN,NaN


In [4]:
# Here we already can check a datatype error: date and delivery_hour columns are set as "object" and not time and datetime
# Other mistake is the labeling of the last three rows, they must be clearer

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10161 entries, 0 to 10160
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             10161 non-null  object 
 1   order_id         10161 non-null  object 
 2   order_amount     10161 non-null  float64
 3   region           10161 non-null  object 
 4   items_delivered  10161 non-null  int64  
 5   items_missing    10161 non-null  int64  
 6   delivery_hour    10161 non-null  object 
 7   customer_id      10161 non-null  object 
 8   customer_name    10161 non-null  object 
 9   customer_age     10161 non-null  int64  
 10  driver_id        10161 non-null  object 
 11  driver_name      10161 non-null  object 
 12  driver_age       10161 non-null  int64  
 13  driver_trips     10161 non-null  int64  
 14  product_name     1662 non-null   object 
 15  category         1662 non-null   object 
 16  price            1662 non-null   float64
dtypes: float64(2

In [5]:
# Another suspect thing happening is that for some registers, the value lost is bigger than the order price
# We can assume that the "order_amount" is the value without the lost product(s) prices
# This is a data we got in the end of the year, the person probably got time to have his refund. That can be a cause
# We'll create an extra column with the real_order_amount

df[df['price'] > 0.1].head(5)

,date,order_id,order_amount,region,items_delivered,items_missing,delivery_hour,customer_id,customer_name,customer_age,driver_id,driver_name,driver_age,driver_trips,product_name,category,price
18,2023-02-13,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,78.28,Clermont,10,2,05:57:03,WCID5065,Bruce Ruiz,48,WDID10064,Charlotte Davila,18,37,Hummus,Pantry,7.92
19,2023-02-13,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,78.28,Clermont,10,2,05:57:03,WCID5065,Bruce Ruiz,48,WDID10064,Charlotte Davila,18,37,Olive Oil,Pantry,6.09
20,2023-02-13,de6343f9-fa09-4cac-8da8-7c2706825e22,133.73,Sanford,13,2,11:20:24,WCID5028,Allison Barker,88,WDID10389,Megan Smith,24,64,Croutons,Pantry,13.31
21,2023-02-13,de6343f9-fa09-4cac-8da8-7c2706825e22,133.73,Sanford,13,2,11:20:24,WCID5028,Allison Barker,88,WDID10389,Megan Smith,24,64,Ground Beef,Pantry,13.27
22,2023-02-13,fe4bffcc-e466-4f05-b111-777a4808abcd,980.99,Sanford,15,1,12:10:18,WCID5145,Joshua Davis,34,WDID10066,Ryan Gomez,18,60,Flour,Pantry,3.48


In [6]:
# Correct date column type 

df['date'] = pd.to_datetime(df['date'])
df['date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10161 entries, 0 to 10160
Series name: date
Non-Null Count  Dtype         
--------------  -----         
10161 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 79.5 KB


In [7]:
# Correct delivery_hour to time

df['delivery_hour'] = pd.to_datetime(df['delivery_hour'], format='%H:%M:%S', errors='coerce').dt.time
type(df['delivery_hour'].iloc[0])

datetime.time

In [8]:
# Rename Last Three Columns

df = df.rename(columns={'product_name': 'missing_product_name', 
                   'category': 'missing_product_category',
                   'price': 'missing_amount'})
df.head()

,date,order_id,order_amount,region,items_delivered,items_missing,delivery_hour,customer_id,customer_name,customer_age,driver_id,driver_name,driver_age,driver_trips,missing_product_name,missing_product_category,missing_amount
0,2023-02-12,b0b0ed0b-6d92-4747-a27d-32fa5b293d48,341.92,Altamonte Springs,19,0,04:02:37,WCID5359,Amanda Frank,48,WDID10634,Kevin Castillo DDS,65,48,NaN,NaN,NaN
1,2023-02-12,052ef301-b6a1-4e08-a6eb-8c92f64c11af,113.50,Clermont,8,0,17:58:42,WCID5000,Dustin Spencer,38,WDID10766,Heather Welch,60,25,NaN,NaN,NaN
2,2023-02-12,1c052a51-448f-4859-a739-7066ac41b8e8,442.30,Apopka,19,0,17:14:01,WCID5040,Amy Harrison,72,WDID10966,Shannon Reyes,43,24,NaN,NaN,NaN
3,2023-02-12,ca4bb404-bb71-40ee-925d-b816c97f4a9c,277.32,Clermont,13,0,03:52:42,WCID5867,Jose Jennings,85,WDID11100,Brett Young,48,39,NaN,NaN,NaN
4,2023-02-12,9cc29314-15ad-4259-9547-7b70177ed6de,462.46,Clermont,15,0,14:01:55,WCID6045,Zachary Campbell,60,WDID09899,Stephen Patton,18,47,NaN,NaN,NaN


In [9]:
# Total orders per client
df_nodup = df.drop_duplicates()
total_orders = df_nodup.groupby('customer_id')['order_id'].count()
df['customer_orders'] = df_nodup['customer_id'].map(total_orders)
df.head()


,date,order_id,order_amount,region,items_delivered,items_missing,delivery_hour,customer_id,customer_name,customer_age,driver_id,driver_name,driver_age,driver_trips,missing_product_name,missing_product_category,missing_amount,customer_orders
0,2023-02-12,b0b0ed0b-6d92-4747-a27d-32fa5b293d48,341.92,Altamonte Springs,19,0,04:02:37,WCID5359,Amanda Frank,48,WDID10634,Kevin Castillo DDS,65,48,NaN,NaN,NaN,10.0
1,2023-02-12,052ef301-b6a1-4e08-a6eb-8c92f64c11af,113.50,Clermont,8,0,17:58:42,WCID5000,Dustin Spencer,38,WDID10766,Heather Welch,60,25,NaN,NaN,NaN,7.0
2,2023-02-12,1c052a51-448f-4859-a739-7066ac41b8e8,442.30,Apopka,19,0,17:14:01,WCID5040,Amy Harrison,72,WDID10966,Shannon Reyes,43,24,NaN,NaN,NaN,11.0
3,2023-02-12,ca4bb404-bb71-40ee-925d-b816c97f4a9c,277.32,Clermont,13,0,03:52:42,WCID5867,Jose Jennings,85,WDID11100,Brett Young,48,39,NaN,NaN,NaN,7.0
4,2023-02-12,9cc29314-15ad-4259-9547-7b70177ed6de,462.46,Clermont,15,0,14:01:55,WCID6045,Zachary Campbell,60,WDID09899,Stephen Patton,18,47,NaN,NaN,NaN,5.0


In [10]:
# Create New dataset with only orders with lost products
df_lost = df[df['missing_amount'] > 0]
df_lost.head()

,date,order_id,order_amount,region,items_delivered,items_missing,delivery_hour,customer_id,customer_name,customer_age,driver_id,driver_name,driver_age,driver_trips,missing_product_name,missing_product_category,missing_amount,customer_orders
18,2023-02-13,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,78.28,Clermont,10,2,05:57:03,WCID5065,Bruce Ruiz,48,WDID10064,Charlotte Davila,18,37,Hummus,Pantry,7.92,7.0
19,2023-02-13,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,78.28,Clermont,10,2,05:57:03,WCID5065,Bruce Ruiz,48,WDID10064,Charlotte Davila,18,37,Olive Oil,Pantry,6.09,7.0
20,2023-02-13,de6343f9-fa09-4cac-8da8-7c2706825e22,133.73,Sanford,13,2,11:20:24,WCID5028,Allison Barker,88,WDID10389,Megan Smith,24,64,Croutons,Pantry,13.31,12.0
21,2023-02-13,de6343f9-fa09-4cac-8da8-7c2706825e22,133.73,Sanford,13,2,11:20:24,WCID5028,Allison Barker,88,WDID10389,Megan Smith,24,64,Ground Beef,Pantry,13.27,12.0
22,2023-02-13,fe4bffcc-e466-4f05-b111-777a4808abcd,980.99,Sanford,15,1,12:10:18,WCID5145,Joshua Davis,34,WDID10066,Ryan Gomez,18,60,Flour,Pantry,3.48,9.0


In [11]:
# Create a new smaller database
df_lost_cost_driv = df_lost[['order_id', 'customer_id', 'customer_name', 'customer_orders', 'driver_id', 'driver_name', 'driver_trips']]

# Remove duplicates
df_lost_cost_driv = df_lost_cost_driv.drop_duplicates()

# Create variables
suspect_customers = df_lost_cost_driv['customer_name'].value_counts()
suspect_drivers = df_lost_cost_driv['driver_name'].value_counts()

# New 2 columns based on previous variables
df_lost_cost_driv['customer_incidence'] = df_lost_cost_driv['customer_name'].map(suspect_customers)
df_lost_cost_driv['driver_incidence'] = df_lost_cost_driv['driver_name'].map(suspect_drivers)

# % of lost orders
df_lost_cost_driv['lost_incidence_driver'] = round(df_lost_cost_driv['driver_incidence'] / df_lost_cost_driv['driver_trips'], 3)
df_lost_cost_driv['lost_incidence_customer'] = round(df_lost_cost_driv['customer_incidence'] / df_lost_cost_driv['customer_orders'], 3)

df_lost_cost_driv.head(5)



,order_id,customer_id,customer_name,customer_orders,driver_id,driver_name,driver_trips,customer_incidence,driver_incidence,lost_incidence_driver,lost_incidence_customer
18,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,WCID5065,Bruce Ruiz,7.0,WDID10064,Charlotte Davila,37,2,4,0.108,0.286
20,de6343f9-fa09-4cac-8da8-7c2706825e22,WCID5028,Allison Barker,12.0,WDID10389,Megan Smith,64,2,4,0.062,0.167
22,fe4bffcc-e466-4f05-b111-777a4808abcd,WCID5145,Joshua Davis,9.0,WDID10066,Ryan Gomez,60,2,4,0.067,0.222
23,849e79f0-13bf-4a80-bc02-62c208ebe4cd,WCID5762,Matthew Osborne,8.0,WDID10730,Kevin Gilmore,11,3,3,0.273,0.375
24,9923a278-e245-4836-aa35-5f6b7f28428f,WCID5228,Mr. Gregory Luna,13.0,WDID10740,Ronald Doyle,78,4,3,0.038,0.308


In [12]:
# Quantile Segmentation for drivers 
df_lost_cost_driv['fraudulent_driver?'] = pd.qcut(df_lost_cost_driv['lost_incidence_driver'], q=5, labels=['Very Unlikely', 'Unlikely', 'Maybe', 'Probable', 'Very Probable'])

# Quantile Segmentation for customers
df_lost_cost_driv['fraudulent_customer?'] = pd.qcut(df_lost_cost_driv['lost_incidence_customer'], q=5, labels=['Very Unlikely', 'Unlikely', 'Maybe', 'Probable', 'Very Probable'])

df_lost_cost_driv.head()




,order_id,customer_id,customer_name,customer_orders,driver_id,driver_name,driver_trips,customer_incidence,driver_incidence,lost_incidence_driver,lost_incidence_customer,fraudulent_driver?,fraudulent_customer?
18,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,WCID5065,Bruce Ruiz,7.0,WDID10064,Charlotte Davila,37,2,4,0.108,0.286,Probable,Probable
20,de6343f9-fa09-4cac-8da8-7c2706825e22,WCID5028,Allison Barker,12.0,WDID10389,Megan Smith,64,2,4,0.062,0.167,Unlikely,Unlikely
22,fe4bffcc-e466-4f05-b111-777a4808abcd,WCID5145,Joshua Davis,9.0,WDID10066,Ryan Gomez,60,2,4,0.067,0.222,Unlikely,Maybe
23,849e79f0-13bf-4a80-bc02-62c208ebe4cd,WCID5762,Matthew Osborne,8.0,WDID10730,Kevin Gilmore,11,3,3,0.273,0.375,Very Probable,Very Probable
24,9923a278-e245-4836-aa35-5f6b7f28428f,WCID5228,Mr. Gregory Luna,13.0,WDID10740,Ronald Doyle,78,4,3,0.038,0.308,Very Unlikely,Probable


In [ ]:
# Reordering the columns
# This will become a new table in our Power BI

cols = [
    'order_id',
    'customer_id',
    'customer_name',
    'customer_orders',
    'customer_incidence',
    'lost_incidence_customer',
    'fraudulent_customer?',
    'driver_id',
    'driver_name',
    'driver_trips',
    'driver_incidence',
    'lost_incidence_driver',
    'fraudulent_driver?'
]

df_lost_cost_driv = df_lost_cost_driv[cols]
df_lost_cost_driv.head()

,order_id,customer_id,customer_name,customer_orders,customer_incidence,lost_incidence_customer,fraudulent_customer?,driver_id,driver_name,driver_trips,driver_incidence,lost_incidence_driver,fraudulent_driver?
18,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,WCID5065,Bruce Ruiz,7.0,2,0.286,Probable,WDID10064,Charlotte Davila,37,4,0.108,Probable
20,de6343f9-fa09-4cac-8da8-7c2706825e22,WCID5028,Allison Barker,12.0,2,0.167,Unlikely,WDID10389,Megan Smith,64,4,0.062,Unlikely
22,fe4bffcc-e466-4f05-b111-777a4808abcd,WCID5145,Joshua Davis,9.0,2,0.222,Maybe,WDID10066,Ryan Gomez,60,4,0.067,Unlikely
23,849e79f0-13bf-4a80-bc02-62c208ebe4cd,WCID5762,Matthew Osborne,8.0,3,0.375,Very Probable,WDID10730,Kevin Gilmore,11,3,0.273,Very Probable
24,9923a278-e245-4836-aa35-5f6b7f28428f,WCID5228,Mr. Gregory Luna,13.0,4,0.308,Probable,WDID10740,Ronald Doyle,78,3,0.038,Very Unlikely
